In [ ]:
!pip install transformers fastapi

In [ ]:
from typing import Optional
from fastapi import FastAPI
from pydantic import BaseModel
import time
from transformers import AutoTokenizer, AutoModel
import torch
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
# stop_words = set(stopwords.words('english'))

In [ ]:
class Messages(BaseModel):
    message_a: str
    message_b: Optional[str] = None

app = FastAPI()

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

cos_dict = {}
time_dict = {}

In [ ]:
@app.get('/')
def index():
    return {'message': 'This is the homepage of the API. Send POST Request to /get_similarity to obtain a cosine distance for message_a and message_b.'}


@app.post("/get_similarity")
def find_similarity(messages: Messages):
  time_start_similarity = time.time()

  messages_id = "some id"
  sentences = [
      messages.message_a,
      messages.message_b
  ]

  tokens = {'input_ids': [], 'attention_mask': []}

  for sentence in sentences:
      
      new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

  outputs = model(**tokens)
  embeddings = outputs.last_hidden_state

  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
  masked_embeddings = embeddings * mask
  summed = torch.sum(masked_embeddings, 1)
  summed_mask = torch.clamp(mask.sum(1), min=1e-9)
  mean_pooled = summed / summed_mask


  mean_pooled = mean_pooled.detach().numpy()
  cosine_array = cosine_similarity(
      [mean_pooled[0]],
      mean_pooled[1:]
  )

  cosine_similarity_value = cosine_array.sum()/(len(sentences)-1)

  time_dict[messages_id] = time.time() - time_start_similarity
  cos_dict[messages_id] = cosine_similarity_value

  return cosine_similarity_value

In [ ]:
## FOR TESTING THE MODEL

test = Messages(message_a = "Once upon a time",
                message_b = "Twice upon a time")

print(find_similarity(test))

0.9999998807907104
